In [1]:
import pandas as pd
import numpy as np
import os
from shutil import copy2

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(),'projections'))
target_path = os.path.abspath(os.path.join(os.getcwd(),'target_predictions'))
if not os.path.exists(target_path):
    os.makedirs(target_path)